In [1]:
import tensorflow as tf

In [78]:
class ModelBuilder(object):

    def __init__(self, model_id):
        self.model_id = model_id

    def get_input(self, mode):
        """Implement in derived class"""
        raise NotImplemented()

    def get_train_inputs(self):
        return self.get_input(tf.estimator.ModeKeys.TRAIN)

    def get_predict_inputs(self):
        return self.get_input(tf.estimator.ModeKeys.PREDICT)

    def base_model(self, features, mode):
        """Implemented in the derived class
        """
        raise NotImplemented()

    def get_train_op(self, loss, step):
        raise NotImplemented()
    
    def get_loss(self, predictions, labels):
        raise NotImplemented()

    def build_estimator(self, features, labels, mode, config=None):
        """This method will build the estimator"""
        predictions = self.base_model(features, mode)
        network_arguments = dict(mode=mode, predictions=predictions)

        if mode == tf.estimator.ModeKeys.PREDICT:
            """Execution pipeline"""
            return tf.estimator.EstimatorSpec(**network_arguments)

        
        loss = self.get_loss(predictions, labels)
        network_arguments['loss'] = loss

        if mode == tf.estimator.ModeKeys.EVAL:
            return tf.estimator.EstimatorSpec(**network_arguments)

        """Will provide current step. In this case the epoch"""
        step = tf.train.get_or_create_global_step()
        """Collection of operations. These are the operations
        that are to be executed after training step"""
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

        with tf.control_dependencies(update_ops):
            train_op = self.get_train_op(loss = loss,
                                         step = step)

        network_arguments['train_op'] = train_op

        if mode == tf.estimator.ModeKeys.TRAIN:
            return tf.estimator.EstimatorSpec(**network_arguments)

    def get_estimator(self, config=None):
        """Get the `tf.estimator.Estimator` defined by this builder."""
        return tf.estimator.Estimator(
            self.build_estimator, self.model_dir, config=config)

    def train(self, config=None, **train_kwargs):
        """Wrapper around `tf.estimator.Estimator.train`."""
        estimator = self.get_estimator(config=config)
        estimator.train(self.get_train_inputs, **train_kwargs)

    def evaluate(self, config=None, **train_kwargs):
        """Wrapper around `tf.estimator.Estimator.train`."""
        estimator = self.get_estimator(config=config)
        estimator.evaluate(self.get_predict_inputs, **train_kwargs)

In [97]:
import os

from sklearn.datasets import fetch_olivetti_faces
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

def initialize_datasets():
    data = fetch_olivetti_faces()
    targets = data.target
    features = data.images
    encoder = OneHotEncoder()
    targets = encoder.fit_transform(targets.reshape(-1, 1)).todense()
    xTrain, xTest, yTrain, yTest = train_test_split(features, targets, test_size=0.1)

    return {'train':{'features': xTrain, 'labels' : yTrain}, 'test':{'features': xTest, 'labels' : yTest}}

class Classifier(ModelBuilder):
    
    
    def __init__(self, model_id):
        super(Classifier, self).__init__(model_id)
        self.data_dictionary = initialize_datasets()
    
    def make_dataset(self, features, labels):
        dataset = tf.data.Dataset.from_tensor_slices((features, labels))
        return dataset
    
    def get_input(self, mode):
        
        if mode == tf.estimator.ModeKeys.TRAIN:
            dataset = self.make_dataset(self.data_dictionary['train']['features'].reshape(360, -1),
                                   self.data_dictionary['train']['labels'].reshape(360, -1))
        else:
            dataset = self.make_dataset(self.data_dictionary['test']['features'],
                                   self.data_dictionary['test']['labels'])
        
        dataset = dataset.repeat()
        dataset = dataset.prefetch(2)

        return dataset.make_one_shot_iterator().get_next()
    
    def base_model(self, features, mode):
        features = tf.reshape(features, [1, 64 * 64])
        predictions = tf.nn.softmax(tf.matmul(features, tf.Variable(tf.zeros([64 * 64, 40]))) + tf.Variable(tf.zeros([40])))
        
        return predictions
    
    def get_loss(self, predictions, labels):
        labels = tf.cast(labels, tf.float32)
        loss_function = tf.reduce_mean(-tf.reduce_sum(labels * tf.log(predictions), reduction_indices=1))
        return loss_function
    
    def get_train_op(self, loss, step):
        optimizer = tf.train.AdamOptimizer()
        train_operation = optimizer.minimize(loss, step)
        return train_operation
    
    @property
    def model_dir(self):
        return os.path.join(os.getcwd(), "model", self.model_id)

In [102]:
classifier = Classifier("12")
classifier.train(max_steps=5000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/home/saurabh/Documents/tfMulti/model/12', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd7b4f0dac8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.


/home/saurabh/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /home/saurabh/Documents/tfMulti/model/12/model.ckpt.
INFO:tensorflow:loss = 3.6888795, step = 1
INFO:tensorflow:global_step/sec: 759.56
INFO:tensorflow:loss = 1.3600674, step = 101 (0.132 sec)
INFO:tensorflow:global_step/sec: 1420.25
INFO:tensorflow:loss = 13.213498, step = 201 (0.070 sec)
INFO:tensorflow:global_step/sec: 1235.65
INFO:tensorflow:loss = 0.42529956, step = 301 (0.081 sec)
INFO:tensorflow:global_step/sec: 1426.64
INFO:tensorflow:loss = 7.0349064, step = 401 (0.070 sec)
INFO:tensorflow:global_step/sec: 1449.23
INFO:tensorflow:loss = 13.983682, step = 501 (0.069 sec)
INFO:tensorflow:global_step/sec: 1335.76
INFO:tensorflow:loss = 1.7622852, step = 601 (0.075 sec)
INFO:tensorflow:global_step/sec: 1293.38
INFO:tensorflow:loss = 0.13245636, step = 701 (0.077 sec)
INFO:tensorflow:global_step/sec: 1389.13
INFO:tensorflow:loss = 2.3623645, step = 801 (0